In [1]:
from person2vec.train_embeddings import train

Using TensorFlow backend.


In [ ]:
model = train.train_model(epochs=30)

In [2]:
from person2vec.test_embeddings import test_tasks

In [ ]:
test_tasks.test_model(model)

In [ ]:
model = train.train_model(model=model, epochs=30)

In [ ]:
test_tasks.test_model(model)

In [ ]:
model.save_weights('../person2vec/data/weights/dense1000_60epoch_97genderacc.h5')

In [ ]:
model2 = train._build_default_model()

In [ ]:
model2.load_weights('../person2vec/data/weights/dense1000_60epoch_97genderacc.h5')

In [ ]:
model = train.train_model(model=model2, epochs=1)

In [3]:
from person2vec.generators import training_data_generator

In [4]:
data_gen = training_data_generator.EmbeddingDataGenerator(300,4)

In [5]:
test_tasks.test_word2vec(data_gen.word_vectors, data_gen=data_gen)

Train on 750 samples, validate on 252 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.2367 - acc: 0.5813 - val_loss: 0.2304 - val_acc: 0.5754
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.2126 - acc: 0.6773 - val_loss: 0.2107 - val_acc: 0.6587
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.1952 - acc: 0.7347 - val_loss: 0.1919 - val_acc: 0.7143
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.1803 - acc: 0.7707 - val_loss: 0.1783 - val_acc: 0.7500
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.1673 - acc: 0.8080 - val_loss: 0.1650 - val_acc: 0.8135
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.1558 - acc: 0.8400 - val_loss: 0.1533 - val_acc: 0.8611
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.1460 - acc: 0.8733 - val_loss: 0.1428 - val_acc: 0.8929
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.1369 - acc: 0.8840 - val_l

750/750 [==============================] - 0s - loss: 0.0371 - acc: 0.9760 - val_loss: 0.0596 - val_acc: 0.9405
Epoch 67/90
750/750 [==============================] - 0s - loss: 0.0367 - acc: 0.9773 - val_loss: 0.0593 - val_acc: 0.9405
Epoch 68/90
750/750 [==============================] - 0s - loss: 0.0363 - acc: 0.9787 - val_loss: 0.0592 - val_acc: 0.9405
Epoch 69/90
750/750 [==============================] - 0s - loss: 0.0359 - acc: 0.9800 - val_loss: 0.0590 - val_acc: 0.9405
Epoch 70/90
750/750 [==============================] - 0s - loss: 0.0356 - acc: 0.9800 - val_loss: 0.0590 - val_acc: 0.9405
Epoch 71/90
750/750 [==============================] - 0s - loss: 0.0352 - acc: 0.9800 - val_loss: 0.0587 - val_acc: 0.9405
Epoch 72/90
750/750 [==============================] - 0s - loss: 0.0349 - acc: 0.9800 - val_loss: 0.0584 - val_acc: 0.9405
Epoch 73/90
750/750 [==============================] - 0s - loss: 0.0346 - acc: 0.9800 - val_loss: 0.0585 - val_acc: 0.9405
Epoch 74/90
750/750 

In [6]:
import pickle

In [7]:
embeds = pickle.load(open("../person2vec/data/weights/embed_weights_1.pk", "rb" ))

In [8]:
test_tasks.test_embeddings(embeds, data_gen=data_gen)

Train on 750 samples, validate on 252 samples
Epoch 1/90
750/750 [==============================] - 0s - loss: 0.2403 - acc: 0.5613 - val_loss: 0.1836 - val_acc: 0.8016
Epoch 2/90
750/750 [==============================] - 0s - loss: 0.1393 - acc: 0.9040 - val_loss: 0.1141 - val_acc: 0.9365
Epoch 3/90
750/750 [==============================] - 0s - loss: 0.0918 - acc: 0.9627 - val_loss: 0.0834 - val_acc: 0.9603
Epoch 4/90
750/750 [==============================] - 0s - loss: 0.0690 - acc: 0.9680 - val_loss: 0.0674 - val_acc: 0.9603
Epoch 5/90
750/750 [==============================] - 0s - loss: 0.0564 - acc: 0.9760 - val_loss: 0.0584 - val_acc: 0.9683
Epoch 6/90
750/750 [==============================] - 0s - loss: 0.0485 - acc: 0.9760 - val_loss: 0.0517 - val_acc: 0.9683
Epoch 7/90
750/750 [==============================] - 0s - loss: 0.0428 - acc: 0.9760 - val_loss: 0.0475 - val_acc: 0.9683
Epoch 8/90
750/750 [==============================] - 0s - loss: 0.0384 - acc: 0.9773 - val_l

750/750 [==============================] - 0s - loss: 0.0079 - acc: 0.9947 - val_loss: 0.0183 - val_acc: 0.9762
Epoch 67/90
750/750 [==============================] - 0s - loss: 0.0078 - acc: 0.9947 - val_loss: 0.0183 - val_acc: 0.9762
Epoch 68/90
750/750 [==============================] - 0s - loss: 0.0077 - acc: 0.9947 - val_loss: 0.0181 - val_acc: 0.9762
Epoch 69/90
750/750 [==============================] - 0s - loss: 0.0076 - acc: 0.9947 - val_loss: 0.0180 - val_acc: 0.9762
Epoch 70/90
750/750 [==============================] - ETA: 0s - loss: 0.0077 - acc: 0.994 - 0s - loss: 0.0075 - acc: 0.9947 - val_loss: 0.0180 - val_acc: 0.9762
Epoch 71/90
750/750 [==============================] - 0s - loss: 0.0074 - acc: 0.9947 - val_loss: 0.0180 - val_acc: 0.9762
Epoch 72/90
750/750 [==============================] - 0s - loss: 0.0074 - acc: 0.9947 - val_loss: 0.0179 - val_acc: 0.9762
Epoch 73/90
750/750 [==============================] - 0s - loss: 0.0072 - acc: 0.9947 - val_loss: 0.0177 

In [ ]:
from keras.layers import Input, Flatten, Dropout, Embedding, Dense
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import optimizers

In [ ]:
input_tensor_words = Input(shape=(32,300,), dtype='float32', name='word_input')
input_tensor_person = Input(shape=(4,), dtype='int32', name='person_input')

#word_embedding_layer = Embedding(10000, 300, input_length=32, name='word_embedding', trainable=False)(input_tensor_words)
word_embedding_layer = Flatten()(input_tensor_words)
word_embedding_layer = Dropout(0.)(word_embedding_layer)


person_embedding_layer = Embedding(1693, 300, input_length=4, name='person_embedding')(input_tensor_person)
person_embedding_layer = Flatten()(person_embedding_layer)
person_embedding_layer = Dropout(0.)(person_embedding_layer)


joint_embeds = Concatenate(name='joint_embeds')([word_embedding_layer, person_embedding_layer])


nex = Dropout(0.)(joint_embeds)
nex = Dense(100, activation="relu", name='dense_consolidator')(nex)
nex = Dropout(0.)(nex)
full_out = Dense (4, activation='softmax', name='final_output')(nex)

model = Model([input_tensor_words, input_tensor_person], full_out) 
model.summary()

In [ ]:
import pandas as pd
my_list = ['b','c','d','e']
my_frame = pd.DataFrame([0,1,2,3])
my_frame[0] = my_frame[0].apply(my_function, args = (my_list,))
my_frame

In [ ]:
my_frame[0].map(x)

In [ ]:
opt = optimizers.adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def my_function(num, my_list):
    return my_list[num]